**Steam Table Simulation: Steam Expansion Process in a Turbine**

In this tutorial, we will simulate a steam expansion process through a turbine using thermodynamic properties obtained from standard steam tables. This approach is commonly used for water and steam systems, where accurate property data (enthalpy, entropy, specific volume) is tabulated based on temperature and pressure.

We assume that the steam turbine operates at steady state.

### Inlet Specifications:

* **Mole Flow Rate**: 100 mol/s
* **Inlet Pressure**: 3 MPa
* **Inlet Temperature**: 773.15 K (500°C)

We will simulate two different cases based on turbine configuration inputs provided by the user:

---

### **Case 1: Fixed Isentropic Efficiency and Pressure Drop**

In this scenario, we assume that the user specifies the desired pressure decrease and the isentropic efficiency of the turbine. Steam properties at inlet and isentropic outlet conditions will be determined using steam tables.

* **Pressure Drop**: 2 MPa
* **Isentropic Efficiency**: 0.85

---

### **Case 2: Fixed Isentropic Efficiency and Pressure Ratio**

Here, the user defines a pressure ratio instead of a pressure drop. The final pressure is calculated from the inlet pressure and pressure ratio, and steam table data is used to compute the isentropic and actual outlet enthalpy.

* **Pressure Ratio**: 0.5
* **Isentropic Efficiency**: 0.85

---

For both cases, outlet enthalpies and power output from the turbine will be calculated using steam table data. This approach is ideal when working with saturated or superheated steam in power plant simulations or thermodynamic cycle analysis.

---


In [3]:
# Import pyomo package
from pyomo.environ import ConcreteModel, Constraint, value, units

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the IAPWS property package to create a properties block for steam in the flowsheet
from idaes.models.properties import iapws95

from idaes.models.properties.iapws95 import htpx

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the default IPOPT solver
from idaes.core.solvers import get_solver

# Import a compressor unit
from idaes.models.unit_models.pressure_changer import Turbine, ThermodynamicAssumption

# Import a feed and product stream
from idaes.models.unit_models import Feed, Product

## Setting up the flowsheet

Creating a `ConcreteModel` foundation, attaching the steady-state flowsheet, and declaring the property package for the steam turbine simulation.

In [4]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False)

m.fs.properties = iapws95.Iapws95ParameterBlock()

In [ ]:
m.fs.steam_in = Feed(property_package=m.fs.properties)
m.fs.steam_out = Product(property_package=m.fs.properties)
m.fs.turbine_1 = PressureChanger(
    dynamic=False,
    property_package=m.fs.properties,
    compressor=True,
    thermodynamic_assumption=ThermodynamicAssumption.isentropic
)

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 5


## Connecting the unit operations

In [7]:
# Import necessary Pyomo components
from pyomo.environ import TransformationFactory

# Import the Arc component for connecting unit models
from pyomo.network import Arc

In [8]:
m.fs.s01 = Arc(source=m.fs.steam_in.outlet, destination = m.fs.compressor_1.inlet)
m.fs.s02 = Arc(source=m.fs.compressor_1.outlet, destination = m.fs.steam_out.inlet)

TransformationFactory("network.expand_arcs").apply_to(m)

# Call the degrees_of_freedom function, get initial DOF
degrees_of_freedom(m)

5

## Setting up the conditions

In [9]:
# Fix the stream inlet conditions
m.fs.compressor_1.inlet.flow_mol[0].fix(100) # mol/s

# Use htpx method to obtain the molar enthalpy of inlet stream at the given temperature and pressure conditions 
m.fs.compressor_1.inlet.enth_mol[0].fix(value(htpx(T=308.15*units.K, P=101325*units.Pa))) # T in K, P in Pa
m.fs.compressor_1.inlet.pressure[0].fix(101325)

## Fixing the pressure change and isentropic efficiency

In [10]:
# Fix compressor conditions
m.fs.compressor_1.deltaP.fix(101325*5)
m.fs.compressor_1.efficiency_isentropic.fix(0.85)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The final DOF is {0}".format(DOF_final))

The final DOF is 0


## Initaialization

In [11]:
# Initialize the flowsheet, and set the output at INFO level
m.fs.compressor_1.initialize(outlvl=idaeslog.INFO)
m.fs.steam_in.initialize(outlvl=idaeslog.INFO)
m.fs.steam_out.initialize(outlvl=idaeslog.INFO)

2025-05-12 19:12:52 [INFO] idaes.init.fs.compressor_1: Initialization Complete: optimal - Optimal Solution Found
2025-05-12 19:12:52 [INFO] idaes.init.fs.steam_in: Initialization Complete.
2025-05-12 19:12:52 [INFO] idaes.init.fs.steam_out: Initialization Complete.


## Solve the model

In [12]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
from pyomo.environ import SolverFactory
opt = SolverFactory('ipopt')
solve_status = opt.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [13]:
# View results
m.fs.compressor_1.report()


Unit : fs.compressor_1                                                     Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                   : Value      : Units         : Fixed : Bounds
    Isentropic Efficiency :    0.85000 : dimensionless :  True : (None, None)
          Mechanical Work :     1080.1 :          watt : False : (None, None)
          Pressure Change : 5.0662e+05 :        pascal :  True : (None, None)
           Pressure Ratio :     6.0000 : dimensionless : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                         Units           Inlet     Outlet  
    Molar Flow          mole / second     100.00     100.00
    Mass Flow       kilogram / second     1.8015     1.8015
    T                          kelvin     308.15     308.18
    P                          pascal 1.0132e+05 6.0795e+05
   

In [14]:
m.fs.visualize("Compressor Model")

2025-05-12 19:12:54 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-05-12 19:12:54 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'Compressor Model.json'
2025-05-12 19:12:54 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'Compressor Model.json' in current directory (/home/viraj/Documents/Github/IDAES PSE Examples/Compressor)
2025-05-12 19:12:54 [WARNING] idaes.idaes_ui.fv.fsvis: Flowsheet name changed: old='Compressor Model' new='Compressor-Model'
Flowsheet name changed to 'Compressor-Model'
2025-05-12 19:12:54 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:39697/app?id=Compressor-Model


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x79b964961100>, port=39697, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x79b9653360c0>, save_diagram=<bound method SaveDiagramScreenshot.save_diagram_screenshot of <idaes_ui.fv.save_diagram_screenshot.SaveDiagramScreenshot object at 0x79b965b4e780>>)